In [7]:
import xarray as xr
import numpy as np
import glob
import os

# ============================
# Input folder with Aqua & Terra
# ============================
folder = r"E:\INCOIS\ITCOO\Classes\SNOM_Navy_2025\Chlorophyll\20241015"
out_file = os.path.join(folder, "MODIS_Merged_Chlorophyll_20241015.nc")

# ============================
# Find Aqua & Terra files
# ============================
files = glob.glob(os.path.join(folder, "*.nc"))
if len(files) < 2:
    raise ValueError("Need at least two .nc files (Aqua and Terra) in the folder")

print("Merging files:", files)

# ============================
# Open datasets
# ============================
datasets = [xr.open_dataset(f) for f in files]

# Identify chlorophyll variable
chl_var = None
for cand in ["chlor_a", "chl", "CHL", "chlorophyll"]:
    if cand in datasets[0].variables:
        chl_var = cand
        break
if chl_var is None:
    raise ValueError("No chlorophyll variable found in the datasets")

# ============================
# Merge: average valid pixels
# ============================
chl_arrays = [ds[chl_var] for ds in datasets]

# Stack into one array
chl_stack = xr.concat(chl_arrays, dim="sensor")

# Apply mean ignoring NaNs
chl_merged = chl_stack.mean(dim="sensor", skipna=True)

# ============================
# Save merged dataset
# ============================
merged_ds = datasets[0].copy()
merged_ds[chl_var] = chl_merged
merged_ds.to_netcdf(out_file)

print(f"Merged file saved: {out_file}")


Merging files: ['E:\\INCOIS\\ITCOO\\Classes\\SNOM_Navy_2025\\Chlorophyll\\20241015\\AQUA_MODIS.20241015.L3m.DAY.CHL.chlor_a.4km.nc', 'E:\\INCOIS\\ITCOO\\Classes\\SNOM_Navy_2025\\Chlorophyll\\20241015\\TERRA_MODIS.20241015.L3m.DAY.CHL.chlor_a.4km.nc']
Merged file saved: E:\INCOIS\ITCOO\Classes\SNOM_Navy_2025\Chlorophyll\20241015\MODIS_Merged_Chlorophyll_20241015.nc


In [15]:
import xarray as xr
import numpy as np
import glob
import os

# ============================
# Input folder (all sensors same day)
# ============================
folder = r"E:\INCOIS\ITCOO\Classes\SNOM_Navy_2025\Chlorophyll\20241015"
out_file = os.path.join(folder, "MultiSensor_Merged_Chlorophyll_20241015.nc")

# ============================
# Find all .nc files
# ============================
files = glob.glob(os.path.join(folder, "*.nc"))
if len(files) < 2:
    raise ValueError("Need at least 2 sensor .nc files (Aqua, Terra, VIIRS, OLCI, etc.)")

print("Merging files:", [os.path.basename(f) for f in files])

# ============================
# Open datasets
# ============================
datasets = [xr.open_dataset(f) for f in files]

# ============================
# Identify chlorophyll variable name
# ============================
def find_chl_var(ds):
    for cand in ["chlor_a", "chl", "CHL", "chlorophyll"]:
        if cand in ds.variables:
            return cand
    # fallback: any var containing "chl"
    for v in ds.variables:
        if "chl" in v.lower():
            return v
    raise ValueError("No chlorophyll variable found in dataset.")

chl_var = find_chl_var(datasets[0])
print(f"Chlorophyll variable detected: {chl_var}")

# ============================
# Regrid if necessary (assumes same 4 km grid)
# ============================
# If lat/lon grids are consistent, this step is trivial.
# If grids differ, you would need interpolation (e.g., xESMF).
# Here we assume same lat/lon resolution.
lons = datasets[0]["lon"] if "lon" in datasets[0] else datasets[0]["longitude"]
lats = datasets[0]["lat"] if "lat" in datasets[0] else datasets[0]["latitude"]

chl_arrays = []
for ds in datasets:
    var = find_chl_var(ds)
    chl = ds[var]

    # Mask fill/missing values
    fillv = chl.attrs.get("_FillValue", None)
    missv = chl.attrs.get("missing_value", None)
    chl = chl.where(~np.isclose(chl, fillv)) if fillv is not None else chl
    chl = chl.where(~np.isclose(chl, missv)) if missv is not None else chl

    chl_arrays.append(chl)

# ============================
# Stack and average
# ============================
chl_stack = xr.concat(chl_arrays, dim="sensor")
chl_merged = chl_stack.mean(dim="sensor", skipna=True)

# ============================
# Build merged dataset
# ============================
merged_ds = datasets[0][[v for v in datasets[0].variables if v not in [chl_var]]]  # copy metadata except chl
merged_ds[chl_var] = chl_merged
merged_ds.attrs["description"] = "Merged chlorophyll product from multiple sensors (Aqua MODIS, Terra MODIS, VIIRS, OLCI, etc.)"
merged_ds.attrs["method"] = "Pixel-wise mean of valid values across sensors, skipping NaNs."

# Save output
merged_ds.to_netcdf(out_file)
print(f"Merged file saved: {out_file}")


Merging files: ['AQUA_MODIS.20241015.L3m.DAY.CHL.chlor_a.4km.nc', 'JPSS1_VIIRS.20241015.L3m.DAY.CHL.chlor_a.4km.nc', 'S3A_OLCI_ERRNT.20241015.L3m.DAY.CHL.chlor_a.4km.nc', 'S3B_OLCI_ERRNT.20241015.L3m.DAY.CHL.chlor_a.4km.nc', 'SNPP_VIIRS.20241015.L3m.DAY.CHL.chlor_a.4km.nc', 'TERRA_MODIS.20241015.L3m.DAY.CHL.chlor_a.4km.nc']
Chlorophyll variable detected: chlor_a
Merged file saved: E:\INCOIS\ITCOO\Classes\SNOM_Navy_2025\Chlorophyll\20241015\MultiSensor_Merged_Chlorophyll_20241015.nc
